In [31]:
# May not need this - this is only if you get mem errors

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('notebook', {'NotebookApp': {'iopub_data_rate_limit': 10000000}})  # Adjust the value as needed


{'Notebook': {'Header': True},
 'NotebookApp': {'iopub_data_rate_limit': 10000000}}

In [35]:
import pandas as pd
import numpy as np
from pathlib import Path

import re
import os
import logging

from langchain.document_loaders import TextLoader
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.prompts import(
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
    MessagesPlaceholder)


import streamlit as st


import pinecone



from dotenv import load_dotenv
load_dotenv()







True

In [36]:
# get environment variables and keys
PINE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_ENV = os.getenv('PINECONE_ENV')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [37]:
# Read in scraped data from csv file
rawRecipes_data = Path("Resources/cleaned_recipes_master_CS.csv")

# Read our Crowdfunding data into pandas
rawRecipes_df = pd.read_csv(rawRecipes_data, dtype=object)
rawRecipes_df.head()

,name,url,ingredient_1,ingredient_2,ingredient_3,ingredient_4,ingredient_5,ingredient_6,ingredient_7,ingredient_8,...,ingredient_67,ingredient_68,ingredient_69,ingredient_70,ingredient_71,ingredient_72,ingredient_73,ingredient_74,ingredient_75,ingredient_76
0,"""A Bologna"" - Calamari, Scallops and Clams wit...",https://www.foodnetwork.com/recipes/a-bologna-...,"2 pounds fingerling potatoes, halved lengthwise","7 tablespoons olive oil, divided",Salt and freshly ground black pepper,1/2 bottle Riesling wine,"2 shallots, minced",1 pint heavy cream,20 manilla clams,2 tablespoons butter,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"""A Piece of Turkey"" Easy Roasted Thanksgiving ...",https://www.foodnetwork.com/recipes/jeff-mauro...,One 14-pound turkey,3/4 cup kosher salt,"2 stalks celery, roughly chopped","1 yellow onion, cut into thick slices","1 container fresh poultry herbs (thyme, rosema...","1/2 bulb of garlic, cloves intact",Freshly cracked black pepper,"8 tablespoons (1 stick) unsalted butter, cut i...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"""A Star is Born"" Cupcakes",https://www.foodnetwork.com/recipes/a-star-is-...,Nonstick oil spray,1 cup roasted hazelnuts,4 tablespoons dark brown sugar,16 gingersnaps,"1 stick salted butter, at room temperature","4 tablespoons hazlenut liqueur, such as Frange...",1 1/2 cups all-purpose flour,1 teaspoon baking powder,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"""Aarti Party"" aka Savory Sling",https://www.foodnetwork.com/recipes/aarti-part...,"Ice, as needed","2 ounces vodka, such as Zubrowka","3/4 ounce Pineapple Reduction, recipe follows","3/4 ounce strawberry pickling liquid, from Pic...",3/4 ounce ginger sake,"Soda water, for topping","5 Pickled Strawberries, recipe follows","17 ounces orange-flavored liqueur, such as Coi...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"""Acqua Pazza"" with Sea Bass and Clams",https://www.foodnetwork.com/recipes/acqua-pazz...,8 tablespoons olive oil,"1 large onion, thinly sliced",1 tablespoon minced garlic,2 cups tomato concasse,1/4 teaspoon cayenne pepper,Generous pinch saffron threads,"1 pound Littleneck clams, rinsed several times...","1 pound black mussels, debearded, rinsed sever...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
# clean up some special characters from the data
characters_to_drop = ['Ã±', 'ñ', 'Ã', 'á', '€', 'è', '±','°','©']

# Combine characters to drop into a regex pattern
pattern = '|'.join(map(re.escape, characters_to_drop))

filteredRecipes_df = rawRecipes_df[~rawRecipes_df['name'].str.contains(pattern)]

filteredRecipes_df.head()

,name,url,ingredient_1,ingredient_2,ingredient_3,ingredient_4,ingredient_5,ingredient_6,ingredient_7,ingredient_8,...,ingredient_67,ingredient_68,ingredient_69,ingredient_70,ingredient_71,ingredient_72,ingredient_73,ingredient_74,ingredient_75,ingredient_76
0,"""A Bologna"" - Calamari, Scallops and Clams wit...",https://www.foodnetwork.com/recipes/a-bologna-...,"2 pounds fingerling potatoes, halved lengthwise","7 tablespoons olive oil, divided",Salt and freshly ground black pepper,1/2 bottle Riesling wine,"2 shallots, minced",1 pint heavy cream,20 manilla clams,2 tablespoons butter,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"""A Piece of Turkey"" Easy Roasted Thanksgiving ...",https://www.foodnetwork.com/recipes/jeff-mauro...,One 14-pound turkey,3/4 cup kosher salt,"2 stalks celery, roughly chopped","1 yellow onion, cut into thick slices","1 container fresh poultry herbs (thyme, rosema...","1/2 bulb of garlic, cloves intact",Freshly cracked black pepper,"8 tablespoons (1 stick) unsalted butter, cut i...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"""A Star is Born"" Cupcakes",https://www.foodnetwork.com/recipes/a-star-is-...,Nonstick oil spray,1 cup roasted hazelnuts,4 tablespoons dark brown sugar,16 gingersnaps,"1 stick salted butter, at room temperature","4 tablespoons hazlenut liqueur, such as Frange...",1 1/2 cups all-purpose flour,1 teaspoon baking powder,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"""Aarti Party"" aka Savory Sling",https://www.foodnetwork.com/recipes/aarti-part...,"Ice, as needed","2 ounces vodka, such as Zubrowka","3/4 ounce Pineapple Reduction, recipe follows","3/4 ounce strawberry pickling liquid, from Pic...",3/4 ounce ginger sake,"Soda water, for topping","5 Pickled Strawberries, recipe follows","17 ounces orange-flavored liqueur, such as Coi...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"""Acqua Pazza"" with Sea Bass and Clams",https://www.foodnetwork.com/recipes/acqua-pazz...,8 tablespoons olive oil,"1 large onion, thinly sliced",1 tablespoon minced garlic,2 cups tomato concasse,1/4 teaspoon cayenne pepper,Generous pinch saffron threads,"1 pound Littleneck clams, rinsed several times...","1 pound black mussels, debearded, rinsed sever...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# fill null fields with value of None
filteredRecipes_df = filteredRecipes_df.fillna('None')

In [30]:
filteredRecipes_df.head()

,name,url,ingredient_1,ingredient_2,ingredient_3,ingredient_4,ingredient_5,ingredient_6,ingredient_7,ingredient_8,...,ingredient_67,ingredient_68,ingredient_69,ingredient_70,ingredient_71,ingredient_72,ingredient_73,ingredient_74,ingredient_75,ingredient_76
0,"""A Bologna"" - Calamari, Scallops and Clams wit...",https://www.foodnetwork.com/recipes/a-bologna-...,"2 pounds fingerling potatoes, halved lengthwise","7 tablespoons olive oil, divided",Salt and freshly ground black pepper,1/2 bottle Riesling wine,"2 shallots, minced",1 pint heavy cream,20 manilla clams,2 tablespoons butter,...,None,None,None,None,None,None,None,None,None,None
1,"""A Piece of Turkey"" Easy Roasted Thanksgiving ...",https://www.foodnetwork.com/recipes/jeff-mauro...,One 14-pound turkey,3/4 cup kosher salt,"2 stalks celery, roughly chopped","1 yellow onion, cut into thick slices","1 container fresh poultry herbs (thyme, rosema...","1/2 bulb of garlic, cloves intact",Freshly cracked black pepper,"8 tablespoons (1 stick) unsalted butter, cut i...",...,None,None,None,None,None,None,None,None,None,None
2,"""A Star is Born"" Cupcakes",https://www.foodnetwork.com/recipes/a-star-is-...,Nonstick oil spray,1 cup roasted hazelnuts,4 tablespoons dark brown sugar,16 gingersnaps,"1 stick salted butter, at room temperature","4 tablespoons hazlenut liqueur, such as Frange...",1 1/2 cups all-purpose flour,1 teaspoon baking powder,...,None,None,None,None,None,None,None,None,None,None
3,"""Aarti Party"" aka Savory Sling",https://www.foodnetwork.com/recipes/aarti-part...,"Ice, as needed","2 ounces vodka, such as Zubrowka","3/4 ounce Pineapple Reduction, recipe follows","3/4 ounce strawberry pickling liquid, from Pic...",3/4 ounce ginger sake,"Soda water, for topping","5 Pickled Strawberries, recipe follows","17 ounces orange-flavored liqueur, such as Coi...",...,None,None,None,None,None,None,None,None,None,None
4,"""Acqua Pazza"" with Sea Bass and Clams",https://www.foodnetwork.com/recipes/acqua-pazz...,8 tablespoons olive oil,"1 large onion, thinly sliced",1 tablespoon minced garlic,2 cups tomato concasse,1/4 teaspoon cayenne pepper,Generous pinch saffron threads,"1 pound Littleneck clams, rinsed several times...","1 pound black mussels, debearded, rinsed sever...",...,None,None,None,None,None,None,None,None,None,None


In [10]:
# replace some unicode break spaces
filteredRecipes_df.replace({'\u202f': ''}, regex=True)

,name,url,ingredient_1,ingredient_2,ingredient_3,ingredient_4,ingredient_5,ingredient_6,ingredient_7,ingredient_8,...,ingredient_67,ingredient_68,ingredient_69,ingredient_70,ingredient_71,ingredient_72,ingredient_73,ingredient_74,ingredient_75,ingredient_76
0,"""A Bologna"" - Calamari, Scallops and Clams wit...",https://www.foodnetwork.com/recipes/a-bologna-...,"2 pounds fingerling potatoes, halved lengthwise","7 tablespoons olive oil, divided",Salt and freshly ground black pepper,1/2 bottle Riesling wine,"2 shallots, minced",1 pint heavy cream,20 manilla clams,2 tablespoons butter,...,None,None,None,None,None,None,None,None,None,None
1,"""A Piece of Turkey"" Easy Roasted Thanksgiving ...",https://www.foodnetwork.com/recipes/jeff-mauro...,One 14-pound turkey,3/4 cup kosher salt,"2 stalks celery, roughly chopped","1 yellow onion, cut into thick slices","1 container fresh poultry herbs (thyme, rosema...","1/2 bulb of garlic, cloves intact",Freshly cracked black pepper,"8 tablespoons (1 stick) unsalted butter, cut i...",...,None,None,None,None,None,None,None,None,None,None
2,"""A Star is Born"" Cupcakes",https://www.foodnetwork.com/recipes/a-star-is-...,Nonstick oil spray,1 cup roasted hazelnuts,4 tablespoons dark brown sugar,16 gingersnaps,"1 stick salted butter, at room temperature","4 tablespoons hazlenut liqueur, such as Frange...",1 1/2 cups all-purpose flour,1 teaspoon baking powder,...,None,None,None,None,None,None,None,None,None,None
3,"""Aarti Party"" aka Savory Sling",https://www.foodnetwork.com/recipes/aarti-part...,"Ice, as needed","2 ounces vodka, such as Zubrowka","3/4 ounce Pineapple Reduction, recipe follows","3/4 ounce strawberry pickling liquid, from Pic...",3/4 ounce ginger sake,"Soda water, for topping","5 Pickled Strawberries, recipe follows","17 ounces orange-flavored liqueur, such as Coi...",...,None,None,None,None,None,None,None,None,None,None
4,"""Acqua Pazza"" with Sea Bass and Clams",https://www.foodnetwork.com/recipes/acqua-pazz...,8 tablespoons olive oil,"1 large onion, thinly sliced",1 tablespoon minced garlic,2 cups tomato concasse,1/4 teaspoon cayenne pepper,Generous pinch saffron threads,"1 pound Littleneck clams, rinsed several times...","1 pound black mussels, debearded, rinsed sever...",...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9205,Dorm Room Hot Dog Bun Bread Pudding,https://www.foodnetwork.com/recipes/robert-irv...,Vegetable spray,"8 hot dog buns, cut into 1-inch cubes","1/3 cup golden raisins, rehydrated","4 tablespoons butter, melted",1 cup sweetened condensed milk,1 teaspoon vanilla extract,2 large eggs,2 tablespoons finely chopped pecans,...,None,None,None,None,None,None,None,None,None,None
9206,Doro Wett,https://www.foodnetwork.com/recipes/doro-wett-...,1 small-size chicken,Juice from 2 limes,Kosher salt and freshly ground black pepper,6 cups chopped red onion,3/4 cup clarified butter,2 tablespoons minced garlic,2 tablespoons peeled and finely chopped fresh ...,3 1/2 tablespoons berbere,...,None,None,None,None,None,None,None,None,None,None
9207,Dorothy Zehnder's Apple Kuchen,https://www.foodnetwork.com/recipes/dorothy-ze...,1/2 cup butter or margarine,"1/4 cup sugar, plus 1/4 cup","1/4 teaspoon vanilla, plus 1/2 teaspoon","1 1/4 cups all-purpose flour, plus 1/3 cup","6 ounces cream cheese, softened",1/4 cup sugar,1 egg,1 1/2 cups peeled and chopped apples,...,None,None,None,None,None,None,None,None,None,None
9208,Dorothy's Applesauce Cupcake with Cinnamon Goa...,https://www.foodnetwork.com/recipes/dorothys-a...,1/2 cup brown sugar,1/2 cup granulated sugar,1/4 cup all-purpose flour,1 teaspoon cinnamon,1 teaspoon nutmeg,"2 large Granny Smith apples, julienned",2 cups granulated sugar,1/2 cup (1 stick) unsalted butter,...,None,None,None,None,None,None,None,None,None,None


In [12]:
# replace more unicode
filteredRecipes_df.replace({'\u02da': ''}, regex=True)

,name,url,ingredient_1,ingredient_2,ingredient_3,ingredient_4,ingredient_5,ingredient_6,ingredient_7,ingredient_8,...,ingredient_67,ingredient_68,ingredient_69,ingredient_70,ingredient_71,ingredient_72,ingredient_73,ingredient_74,ingredient_75,ingredient_76
0,"""A Bologna"" - Calamari, Scallops and Clams wit...",https://www.foodnetwork.com/recipes/a-bologna-...,"2 pounds fingerling potatoes, halved lengthwise","7 tablespoons olive oil, divided",Salt and freshly ground black pepper,1/2 bottle Riesling wine,"2 shallots, minced",1 pint heavy cream,20 manilla clams,2 tablespoons butter,...,None,None,None,None,None,None,None,None,None,None
1,"""A Piece of Turkey"" Easy Roasted Thanksgiving ...",https://www.foodnetwork.com/recipes/jeff-mauro...,One 14-pound turkey,3/4 cup kosher salt,"2 stalks celery, roughly chopped","1 yellow onion, cut into thick slices","1 container fresh poultry herbs (thyme, rosema...","1/2 bulb of garlic, cloves intact",Freshly cracked black pepper,"8 tablespoons (1 stick) unsalted butter, cut i...",...,None,None,None,None,None,None,None,None,None,None
2,"""A Star is Born"" Cupcakes",https://www.foodnetwork.com/recipes/a-star-is-...,Nonstick oil spray,1 cup roasted hazelnuts,4 tablespoons dark brown sugar,16 gingersnaps,"1 stick salted butter, at room temperature","4 tablespoons hazlenut liqueur, such as Frange...",1 1/2 cups all-purpose flour,1 teaspoon baking powder,...,None,None,None,None,None,None,None,None,None,None
3,"""Aarti Party"" aka Savory Sling",https://www.foodnetwork.com/recipes/aarti-part...,"Ice, as needed","2 ounces vodka, such as Zubrowka","3/4 ounce Pineapple Reduction, recipe follows","3/4 ounce strawberry pickling liquid, from Pic...",3/4 ounce ginger sake,"Soda water, for topping","5 Pickled Strawberries, recipe follows","17 ounces orange-flavored liqueur, such as Coi...",...,None,None,None,None,None,None,None,None,None,None
4,"""Acqua Pazza"" with Sea Bass and Clams",https://www.foodnetwork.com/recipes/acqua-pazz...,8 tablespoons olive oil,"1 large onion, thinly sliced",1 tablespoon minced garlic,2 cups tomato concasse,1/4 teaspoon cayenne pepper,Generous pinch saffron threads,"1 pound Littleneck clams, rinsed several times...","1 pound black mussels, debearded, rinsed sever...",...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9205,Dorm Room Hot Dog Bun Bread Pudding,https://www.foodnetwork.com/recipes/robert-irv...,Vegetable spray,"8 hot dog buns, cut into 1-inch cubes","1/3 cup golden raisins, rehydrated","4 tablespoons butter, melted",1 cup sweetened condensed milk,1 teaspoon vanilla extract,2 large eggs,2 tablespoons finely chopped pecans,...,None,None,None,None,None,None,None,None,None,None
9206,Doro Wett,https://www.foodnetwork.com/recipes/doro-wett-...,1 small-size chicken,Juice from 2 limes,Kosher salt and freshly ground black pepper,6 cups chopped red onion,3/4 cup clarified butter,2 tablespoons minced garlic,2 tablespoons peeled and finely chopped fresh ...,3 1/2 tablespoons berbere,...,None,None,None,None,None,None,None,None,None,None
9207,Dorothy Zehnder's Apple Kuchen,https://www.foodnetwork.com/recipes/dorothy-ze...,1/2 cup butter or margarine,"1/4 cup sugar, plus 1/4 cup","1/4 teaspoon vanilla, plus 1/2 teaspoon","1 1/4 cups all-purpose flour, plus 1/3 cup","6 ounces cream cheese, softened",1/4 cup sugar,1 egg,1 1/2 cups peeled and chopped apples,...,None,None,None,None,None,None,None,None,None,None
9208,Dorothy's Applesauce Cupcake with Cinnamon Goa...,https://www.foodnetwork.com/recipes/dorothys-a...,1/2 cup brown sugar,1/2 cup granulated sugar,1/4 cup all-purpose flour,1 teaspoon cinnamon,1 teaspoon nutmeg,"2 large Granny Smith apples, julienned",2 cups granulated sugar,1/2 cup (1 stick) unsalted butter,...,None,None,None,None,None,None,None,None,None,None


In [40]:
# write dataframe out to new csv
filteredRecipes_df.to_csv('Resources/filteredRecipes.csv', index=False) 

In [41]:
# load cleaned csv data into documents
loader = TextLoader('Resources/filteredRecipes.csv', encoding='utf-8')
documents = loader.load()            

In [19]:
print(len(documents))

1


In [42]:
#split the text into chunks of 1200 with overlap
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=30)
texts = text_splitter.split_documents(documents)

In [24]:
print(texts[2])

page_content='"""A Piece of Turkey"" Easy Roasted Thanksgiving Turkey",https://www.foodnetwork.com/recipes/jeff-mauro/a-piece-of-turkey-easy-roasted-thanksgiving-turkey-3170836,One 14-pound turkey,3/4 cup kosher salt,"2 stalks celery, roughly chopped\xa0","1 yellow onion, cut into thick slices\xa0","1 container fresh poultry herbs (thyme, rosemary, sage, etc.)","1/2 bulb of garlic, cloves intact\xa0",Freshly cracked black pepper\xa0,"8 tablespoons (1 stick) unsalted butter, cut into tablespoon-size pats",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None' metadata={'source': 'Resources/filteredRecipes.csv'}


In [25]:
print(len(texts))

9435


In [43]:
#embed the data
embeddings = OpenAIEmbeddings()
doc_vectors = embeddings.embed_documents([t.page_content for t in texts[:5]])
doc_vectors

C:\Users\Owner\anaconda3\envs\dev\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


[[-0.012338195013107787,
  -0.01332496667252947,
  -0.010499534211609134,
  -0.007929668772708662,
  -0.003350764051968339,
  0.023128790812749406,
  -0.0019273993120059985,
  -0.0074185351943302225,
  -0.015149429461358561,
  -0.019025525104374903,
  0.00589578320152772,
  0.014652494361310973,
  -0.03373481337900928,
  0.015575374498058334,
  -0.003347214548800948,
  0.021027464448557395,
  0.025258515064925696,
  0.0022575063662023565,
  0.0009299790266668208,
  -0.027601210438468006,
  -0.019735433188433665,
  0.005661513664173489,
  0.005054542336122279,
  0.011450810373695622,
  -0.024193653870160127,
  0.01121654083634139,
  0.02278803664603474,
  -0.03276933734160808,
  -0.01190515143573452,
  0.015759950525407806,
  -0.004713786772423748,
  0.004614399752414231,
  -0.025712855195642016,
  -0.02334176379676058,
  0.00034674551036457,
  0.004564706242409472,
  -0.011181045339006195,
  -0.00708842790730322,
  -0.005764450420181042,
  -0.02131142842723896,
  0.011237838321007023,


In [27]:
print(doc_vectors[0])

[-0.012338195013107787, -0.01332496667252947, -0.010499534211609134, -0.007929668772708662, -0.003350764051968339, 0.023128790812749406, -0.0019273993120059985, -0.0074185351943302225, -0.015149429461358561, -0.019025525104374903, 0.00589578320152772, 0.014652494361310973, -0.03373481337900928, 0.015575374498058334, -0.003347214548800948, 0.021027464448557395, 0.025258515064925696, 0.0022575063662023565, 0.0009299790266668208, -0.027601210438468006, -0.019735433188433665, 0.005661513664173489, 0.005054542336122279, 0.011450810373695622, -0.024193653870160127, 0.01121654083634139, 0.02278803664603474, -0.03276933734160808, -0.01190515143573452, 0.015759950525407806, -0.004713786772423748, 0.004614399752414231, -0.025712855195642016, -0.02334176379676058, 0.00034674551036457, 0.004564706242409472, -0.011181045339006195, -0.00708842790730322, -0.005764450420181042, -0.02131142842723896, 0.011237838321007023, 0.0030011346139360096, 0.0008017519732801176, 0.00540949637815166, -0.03203103323

In [44]:
#write the data to Pinecone vector db into the recipechat index
index_name = "recipechat"

docsearch = PineconeVectorStore.from_documents(texts, embeddings, index_name=index_name)

**Rest is just queries for testing vector data

In [29]:
query = "world famous steak rub from mesa"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

"Bobby's World Famous Steak Rub from Mesa, Bar Americain and Bf Steak",https://www.foodnetwork.com/recipes/bobby-flay/bobbys-world-famous-steak-rub-from-mesa-bar-americain-and-bf-steak-recipe-1940318,1 cup ancho chili powder,1/3 cup Spanish paprika,3 tablespoons ground dried oregano,3 tablespoons ground coriander,3 tablespoons dry mustard,1 tablespoon ground cumin,1 tablespoon kosher salt,1 tablespoon ground black pepper,"2 teaspoons chile de arbol, optional",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [30]:
query = "world famous steak rub from mesa"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

"Bobby's World Famous Steak Rub from Mesa, Bar Americain and Bf Steak",https://www.foodnetwork.com/recipes/bobby-flay/bobbys-world-famous-steak-rub-from-mesa-bar-americain-and-bf-steak-recipe-1940318,1 cup ancho chili powder,1/3 cup Spanish paprika,3 tablespoons ground dried oregano,3 tablespoons ground coriander,3 tablespoons dry mustard,1 tablespoon ground cumin,1 tablespoon kosher salt,1 tablespoon ground black pepper,"2 teaspoons chile de arbol, optional",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [101]:
query = "oregano and dry mustard"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

oregano,1 tablespoon dried thyme,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [31]:
query = "oregano and dry mustard and parsley leaves"
retriever = docsearch.as_retriever(search_type="mmr")
matched_docs = retriever.invoke(query)
for i, d in enumerate(matched_docs):
    print(f"\n## Document {i}\n")
    print(d.page_content)


## Document 0

oregano,1 tablespoon dried thyme,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None

## Document 1

Bob McGee's Herb Oil,https://www.foodnetwork.com/recipes/bob-mcgees-herb-oil-3416206,"1 cup fresh flat parsley leaves, shocked in ice water, drained and patted dry, finely chopped","1 cup fresh oregano leaves, shocked in ice water, drained and patted dry, finely chopped","1/2 cup fresh tarragon leaves, shocked in ice water, drained and patted dry, finely chopped","2 tablespoons brine-packed capers, drained, finely chopped",1 teaspoon chili flakes,"2 salt-packed anchovies, rinsed, dried and finely chopped","1 clove garlic, finely chopped","Zest and juice of 1 lemon, zest finely chopped",2 cups olive oil,Sea salt,Freshly ground black pepper,None,None,None,None,None,Non

In [32]:
query = "oregano and dry mustard and parsley leaves"
found_docs = docsearch.max_marginal_relevance_search(query, k=3, fetch_k=10)
for i, doc in enumerate(found_docs):
    print(f"{i + 1}.", doc.page_content, "\n")

1. oregano,1 tablespoon dried thyme,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None 

2. Bob McGee's Herb Oil,https://www.foodnetwork.com/recipes/bob-mcgees-herb-oil-3416206,"1 cup fresh flat parsley leaves, shocked in ice water, drained and patted dry, finely chopped","1 cup fresh oregano leaves, shocked in ice water, drained and patted dry, finely chopped","1/2 cup fresh tarragon leaves, shocked in ice water, drained and patted dry, finely chopped","2 tablespoons brine-packed capers, drained, finely chopped",1 teaspoon chili flakes,"2 salt-packed anchovies, rinsed, dried and finely chopped","1 clove garlic, finely chopped","Zest and juice of 1 lemon, zest finely chopped",2 cups olive oil,Sea salt,Freshly ground black pepper,None,None,None,None,None,None,None,None,None,None,No